[link](http://hlm.lge.com/issue/browse/HEDATAPLFM-914?attachmentSortBy=dateTime&attachmentOrder=asc)

- 추출 국가 : 필리핀, 호주, 인도네시아, 말레이시아, 싱가포르, 태국, 베트남, 대만, 홍콩, 일본, 인도
- 플랫폼 : webOS22, 23, 24
- 추출 기간 : 24.01.01~25.06.30
-  추출 조건(CN/MI) : 
    - EIM(HDMI) / NL_EIM_INPUTLIST(HDMI)
    - com.palm.app.settings(BT) /	NL_SETTING_SPEAKER(BT)



In [0]:
%python
country_codes = [
    "PH",  # 필리핀
    "AU",  # 호주
    "ID",  # 인도네시아
    "MY",  # 말레이시아
    "SG",  # 싱가포르
    "TH",  # 태국
    "VN",  # 베트남
    "TW",  # 대만
    "HK",  # 홍콩
    "JP",  # 일본
    "IN",  # 인도
]

In [0]:
%sql
select distinct X_Device_Country
from kic_data_ods.tlamp.normal_log_webos24
where 1=1
  and date_ym='2025-06'
  and context_name = 'EIM' and message_id = 'NL_EIM_INPUTLIST'
  and X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')

## EDA

In [0]:
%sql
select 
  -- date_ym, mac_addr, normal_log, X_Device_Country, X_Device_Sales_Model
  date_ym, count(1), count(distinct mac_addr)
from kic_data_ods.tlamp.normal_log_webos24
where 1=1
  and date_ym between '2025-06' and '2025-06'
  and context_name = 'EIM' and message_id = 'NL_EIM_INPUTLIST'
  and X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
group by date_ym
order by date_ym

In [0]:
%sql
select 
    -- date_ym, mac_addr, normal_log, X_Device_Country, X_Device_Sales_Model
    count(1), count(distinct mac_addr)
from kic_data_ods.tlamp.normal_log_webos24
where 1=1
  and date_ym='2025-06'
  and context_name = 'com.palm.app.settings' and message_id = 'NL_SETTING_SPEAKER'
  and X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')

In [0]:
%sql
create table if not exists sandbox.z_yeswook_kim.heds_914_v1
WITH ranked_logs_1 AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_inputlist,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    normal_log AS normal_log_inputlist,
    ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
    AND date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'EIM'
    AND message_id = 'NL_EIM_INPUTLIST'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
),
ranked_logs_2 AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_speaker,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    normal_log AS normal_log_speaker,
    ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
    AND date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'com.palm.app.settings'
    AND message_id = 'NL_SETTING_SPEAKER'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
)
SELECT 
  -- count(1),
  COALESCE(r1.date_ym, r2.date_ym) AS date_ym,
  COALESCE(r1.mac_addr, r2.mac_addr) AS mac_addr,
  r1.log_time_inputlist,
  r2.log_time_speaker,
  COALESCE(r1.X_Device_Country, r2.X_Device_Country) AS X_Device_Country,
  COALESCE(r1.X_Device_Sales_Model, r2.X_Device_Sales_Model) AS X_Device_Sales_Model,
  r1.normal_log_inputlist,
  r2.normal_log_speaker
FROM ranked_logs_1 r1
FULL OUTER JOIN ranked_logs_2 r2
  ON r1.date_ym = r2.date_ym AND r1.mac_addr = r2.mac_addr
WHERE r1.rn = 1 OR r2.rn = 1


In [0]:
%sql
create table if not exists sandbox.z_yeswook_kim.heds_914_v2
WITH ranked_logs_1 AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_inputlist,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    normal_log AS normal_log_inputlist,
    ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn1
  FROM kic_data_ods.tlamp.normal_log_webos23
  WHERE date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'EIM'
    AND message_id = 'NL_EIM_INPUTLIST'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
),
ranked_logs_2 AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_speaker,
    mac_addr,
    X_Device_Country AS X_Device_Country_2,
    X_Device_Sales_Model AS X_Device_Sales_Model_2,
    normal_log AS normal_log_speaker,
    ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn2
  FROM kic_data_ods.tlamp.normal_log_webos23
  WHERE date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'com.palm.app.settings'
    AND message_id = 'NL_SETTING_SPEAKER'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
)

SELECT
  COALESCE(r1.date_ym, r2.date_ym) AS date_ym,
  COALESCE(r1.mac_addr, r2.mac_addr) AS mac_addr,
  COALESCE(r1.X_Device_Country, r2.X_Device_Country_2) AS X_Device_Country,
  COALESCE(r1.X_Device_Sales_Model, r2.X_Device_Sales_Model_2) AS X_Device_Sales_Model,
  r1.log_time_inputlist,
  r1.normal_log_inputlist,
  r2.log_time_speaker,
  r2.normal_log_speaker
FROM ranked_logs_1 r1
FULL OUTER JOIN ranked_logs_2 r2
  ON r1.date_ym = r2.date_ym AND r1.mac_addr = r2.mac_addr
WHERE
  (r1.rn1 = 1 AND r2.rn2 IS NULL) OR  -- r1만 있는 경우
  (r2.rn2 = 1 AND r1.rn1 IS NULL) OR  -- r2만 있는 경우
  (r1.rn1 = 1 AND r2.rn2 = 1)         -- 둘 다 있는 경우


In [0]:
%sql
create table if not exists sandbox.z_yeswook_kim.heds_914_v2
WITH ranked_logs_1 AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_inputlist,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    normal_log AS normal_log_inputlist,
    ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn1
  FROM kic_data_ods.tlamp.normal_log_webos23
  WHERE date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'EIM'
    AND message_id = 'NL_EIM_INPUTLIST'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
),
ranked_logs_2 AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_speaker,
    mac_addr,
    X_Device_Country AS X_Device_Country_2,
    X_Device_Sales_Model AS X_Device_Sales_Model_2,
    normal_log AS normal_log_speaker,
    ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rn2
  FROM kic_data_ods.tlamp.normal_log_webos23
  WHERE date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'com.palm.app.settings'
    AND message_id = 'NL_SETTING_SPEAKER'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
)

SELECT
  COALESCE(r1.date_ym, r2.date_ym) AS date_ym,
  COALESCE(r1.mac_addr, r2.mac_addr) AS mac_addr,
  COALESCE(r1.X_Device_Country, r2.X_Device_Country_2) AS X_Device_Country,
  COALESCE(r1.X_Device_Sales_Model, r2.X_Device_Sales_Model_2) AS X_Device_Sales_Model,
  r1.log_time_inputlist,
  r1.normal_log_inputlist,
  r2.log_time_speaker,
  r2.normal_log_speaker
FROM ranked_logs_1 r1
FULL OUTER JOIN ranked_logs_2 r2
  ON r1.date_ym = r2.date_ym AND r1.mac_addr = r2.mac_addr
WHERE
  (r1.rn1 = 1 AND r2.rn2 IS NULL) OR  -- r1만 있는 경우
  (r2.rn2 = 1 AND r1.rn1 IS NULL) OR  -- r2만 있는 경우
  (r1.rn1 = 1 AND r2.rn2 = 1)         -- 둘 다 있는 경우


In [0]:
%sql
create table if not exists sandbox.z_yeswook_kim.heds_914_v3
WITH ranked_logs_1 AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_inputlist,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    normal_log AS normal_log_inputlist,
    ROW_NUMBER() OVER (PARTITION BY mac_addr ORDER BY log_create_time DESC) AS rn1
  FROM kic_data_ods.tlamp.normal_log_webos23
  WHERE date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'EIM'
    AND message_id = 'NL_EIM_INPUTLIST'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
),
ranked_logs_2 AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_speaker,
    mac_addr,
    normal_log AS normal_log_speaker,
    ROW_NUMBER() OVER (PARTITION BY mac_addr ORDER BY log_create_time DESC) AS rn2
  FROM kic_data_ods.tlamp.normal_log_webos23
  WHERE date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'com.palm.app.settings'
    AND message_id = 'NL_SETTING_SPEAKER'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
)

SELECT
  COALESCE(r1.mac_addr, r2.mac_addr) AS mac_addr,
  r1.date_ym AS date_ym_inputlist,
  r2.date_ym AS date_ym_speaker,
  r1.log_time_inputlist,
  r2.log_time_speaker,
  r1.normal_log_inputlist,
  r2.normal_log_speaker,
  CASE
    WHEN r1.mac_addr IS NOT NULL AND r2.mac_addr IS NOT NULL THEN 'both'
    WHEN r1.mac_addr IS NOT NULL THEN 'only_inputlist'
    WHEN r2.mac_addr IS NOT NULL THEN 'only_speaker'
  END AS source_type
FROM ranked_logs_1 r1
FULL OUTER JOIN ranked_logs_2 r2
  ON r1.mac_addr = r2.mac_addr
WHERE (r1.rn1 = 1 OR r1.rn1 IS NULL)
  AND (r2.rn2 = 1 OR r2.rn2 IS NULL)


In [0]:
%sql
  SELECT 
    count(distinct mac_addr)
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
    FROM kic_data_ods.tlamp.normal_log_webos23
    WHERE date_ym BETWEEN '2025-06' AND '2025-06'
      AND context_name = 'EIM'
      AND message_id = 'NL_EIM_INPUTLIST'
      AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
  ) sub
  WHERE rk = 1

In [0]:
%sql
drop table if exists sandbox.z_yeswook_kim.heds_914_v4;

create table if not exists sandbox.z_yeswook_kim.heds_914_v4
WITH rk1_inputlist AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_inputlist,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    normal_log AS normal_log_inputlist
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
    FROM kic_data_ods.tlamp.normal_log_webos24
    WHERE date_ym BETWEEN '2025-06' AND '2025-06'
      AND context_name = 'EIM'
      AND message_id = 'NL_EIM_INPUTLIST'
      AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
  ) sub
  WHERE rk = 1
),
rk1_speaker AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_speaker,
    mac_addr,
    X_Device_Country AS X_Device_Country_speaker,
    X_Device_Sales_Model AS X_Device_Sales_Model_speaker,
    normal_log AS normal_log_speaker
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
    FROM kic_data_ods.tlamp.normal_log_webos24
    WHERE date_ym BETWEEN '2025-06' AND '2025-06'
      AND context_name = 'com.palm.app.settings'
      AND message_id = 'NL_SETTING_SPEAKER'
      AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
  ) sub
  WHERE rk = 1
)

SELECT
  COALESCE(i.date_ym, s.date_ym) AS date_ym,
  COALESCE(i.mac_addr, s.mac_addr) AS mac_addr,
  COALESCE(i.X_Device_Country, s.X_Device_Country_speaker) AS X_Device_Country,
  COALESCE(i.X_Device_Sales_Model, s.X_Device_Sales_Model_speaker) AS X_Device_Sales_Model,
  i.log_time_inputlist,
  i.normal_log_inputlist,
  s.log_time_speaker,
  s.normal_log_speaker,
  CASE
    WHEN i.mac_addr IS NOT NULL AND s.mac_addr IS NOT NULL THEN 'both'
    WHEN i.mac_addr IS NOT NULL THEN 'only_inputlist'
    WHEN s.mac_addr IS NOT NULL THEN 'only_speaker'
  END AS source_type
FROM rk1_inputlist i
FULL OUTER JOIN rk1_speaker s
  ON i.date_ym = s.date_ym AND i.mac_addr = s.mac_addr


In [0]:
%python
sdf = spark.sql("""
    select 
        *
    from sandbox.z_yeswook_kim.heds_914_v4
""")

In [0]:
def mk_flatten_df(df):
    from pyspark.sql.functions import regexp_extract, from_json, col
    from pyspark.sql.types import StructType, StructField, StringType, ArrayType

    # dev_list 스키마 정의
    inputlist_schema = StructType([
        StructField("dev_list", ArrayType(StructType([
            StructField("name", StringType()),
            StructField("type", StringType()),
            StructField("label", StringType()),
            StructField("spec", StringType()),
            StructField("id", StringType())
        ])))
    ])

    # speaker용 JSON 스키마 정의
    speaker_schema = StructType([
        StructField("type", StringType()),
        StructField("device_name", StringType())
    ])

    # 1. 앞의 빈 JSON 제거 (두 번째 JSON 객체 추출)
    df_cleaned = sdf.withColumn(
        "cleaned_inputlist",
        regexp_extract("normal_log_inputlist", r"(\{.*\}\s*){1}(\{.*\})", 2)
    )

    # 2. JSON 파싱
    df_parsed = df_cleaned.withColumn("parsed_inputlist", from_json(col("cleaned_inputlist"), inputlist_schema))\
                        .withColumn("parsed_speaker", from_json(col("normal_log_speaker"), speaker_schema))


    # 3. 평탄화
    df_flattened = df_parsed.select(
        "date_ym", "mac_addr", "X_Device_Country", "X_Device_Sales_Model",
        "source_type",
        "log_time_inputlist",
        col("parsed_inputlist.dev_list")[0].getItem("name").alias("input_name"),
        col("parsed_inputlist.dev_list")[0].getItem("type").alias("input_type"),
        col("parsed_inputlist.dev_list")[0].getItem("label").alias("input_label"),
        col("parsed_inputlist.dev_list")[0].getItem("spec").alias("input_spec"),
        col("parsed_inputlist.dev_list")[0].getItem("id").alias("input_id"),
        
        "log_time_speaker",
        col("parsed_speaker.type").alias("speaker_type"),
        col("parsed_speaker.device_name").alias("speaker_device_name")
    )

    return df_flattened


In [0]:
display(df_flattened)

In [0]:
%sql
SELECT 
  date_ym,
  mac_addr,
  X_Device_Country,
  X_Device_Sales_Model,
  dev.name AS device_name,
  dev.type AS device_type,
  dev.spec AS device_spec,
  dev.id AS device_id
FROM (
  SELECT 
    date_ym,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    EXPLODE(
      FROM_JSON(
        REGEXP_EXTRACT(normal_log, '\\{.*\\}\\s+(\\{.*\\})', 1),
        'STRUCT<dev_list:ARRAY<STRUCT<name:STRING, type:STRING, label:STRING, spec:STRING, id:STRING>>>'
      ).dev_list
    ) AS dev
  FROM kic_data_ods.tlamp.normal_log_webos23
  WHERE 1=1
    AND date_ym BETWEEN '2025-06' AND '2025-06'
    AND context_name = 'EIM'
    AND message_id = 'NL_EIM_INPUTLIST'
    AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
)


In [0]:
%python
create table if not exists sandbox.z_yeswook_kim.heds_914_v4
WITH rk1_inputlist AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_inputlist,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    normal_log AS normal_log_inputlist
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
    FROM kic_data_ods.tlamp.normal_log_webos24
    WHERE date_ym BETWEEN '2025-06' AND '2025-06'
      AND context_name = 'EIM'
      AND message_id = 'NL_EIM_INPUTLIST'
      AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
  ) sub
  WHERE rk = 1
),
rk1_speaker AS (
  SELECT 
    date_ym,
    log_create_time AS log_time_speaker,
    mac_addr,
    X_Device_Country AS X_Device_Country_speaker,
    X_Device_Sales_Model AS X_Device_Sales_Model_speaker,
    normal_log AS normal_log_speaker
  FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
    FROM kic_data_ods.tlamp.normal_log_webos24
    WHERE date_ym BETWEEN '2025-06' AND '2025-06'
      AND context_name = 'com.palm.app.settings'
      AND message_id = 'NL_SETTING_SPEAKER'
      AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
  ) sub
  WHERE rk = 1
)

SELECT
  COALESCE(i.date_ym, s.date_ym) AS date_ym,
  COALESCE(i.mac_addr, s.mac_addr) AS mac_addr,
  COALESCE(i.X_Device_Country, s.X_Device_Country_speaker) AS X_Device_Country,
  COALESCE(i.X_Device_Sales_Model, s.X_Device_Sales_Model_speaker) AS X_Device_Sales_Model,
  i.log_time_inputlist,
  i.normal_log_inputlist,
  s.log_time_speaker,
  s.normal_log_speaker,
  CASE
    WHEN i.mac_addr IS NOT NULL AND s.mac_addr IS NOT NULL THEN 'both'
    WHEN i.mac_addr IS NOT NULL THEN 'only_inputlist'
    WHEN s.mac_addr IS NOT NULL THEN 'only_speaker'
  END AS source_type
FROM rk1_inputlist i
FULL OUTER JOIN rk1_speaker s
  ON i.date_ym = s.date_ym AND i.mac_addr = s.mac_addr

# 데이터 추출

In [0]:
%python
from datetime import datetime, timedelta

# 시작 월과 종료 월 지정
start_month = "2024-01"
end_month = "2025-06"

# 월 리스트 생성
date_ym_list = []
current = datetime.strptime(start_month, "%Y-%m")
end = datetime.strptime(end_month, "%Y-%m")

while current <= end:
    date_ym_list.append(current.strftime("%Y-%m"))
    # 다음 달로 이동
    current += timedelta(days=32)
    current = current.replace(day=1)


In [0]:
def mk_flatten_df(df):
    from pyspark.sql.functions import regexp_extract, from_json, col
    from pyspark.sql.types import StructType, StructField, StringType, ArrayType

    # dev_list 스키마 정의
    inputlist_schema = StructType([
        StructField("dev_list", ArrayType(StructType([
            StructField("name", StringType()),
            StructField("type", StringType()),
            StructField("label", StringType()),
            StructField("spec", StringType()),
            StructField("id", StringType())
        ])))
    ])

    # speaker용 JSON 스키마 정의
    speaker_schema = StructType([
        StructField("type", StringType()),
        StructField("device_name", StringType())
    ])

    # 1. 앞의 빈 JSON 제거 (두 번째 JSON 객체 추출)
    df_cleaned = df.withColumn(
        "cleaned_inputlist",
        regexp_extract("normal_log_inputlist", r"(\{.*\}\s*){1}(\{.*\})", 2)
    )

    # 2. JSON 파싱
    df_parsed = df_cleaned.withColumn("parsed_inputlist", from_json(col("cleaned_inputlist"), inputlist_schema))\
                        .withColumn("parsed_speaker", from_json(col("normal_log_speaker"), speaker_schema))


    # 3. 평탄화
    df_flattened = df_parsed.select(
        "date_ym", "mac_addr", "X_Device_Country", "X_Device_Sales_Model",
        "source_type",
        "log_time_inputlist",
        col("parsed_inputlist.dev_list")[0].getItem("name").alias("input_name"),
        col("parsed_inputlist.dev_list")[0].getItem("type").alias("input_type"),
        col("parsed_inputlist.dev_list")[0].getItem("label").alias("input_label"),
        col("parsed_inputlist.dev_list")[0].getItem("spec").alias("input_spec"),
        col("parsed_inputlist.dev_list")[0].getItem("id").alias("input_id"),
        
        "log_time_speaker",
        col("parsed_speaker.type").alias("speaker_type"),
        col("parsed_speaker.device_name").alias("speaker_device_name")
    )

    return df_flattened


In [0]:
%python
# 테이블을 DataFrame으로 로드
for webos_version in ['webos22','webos23','webos24'][::]:
  for date_ym in date_ym_list[::]:
    print(webos_version, date_ym,'started')
    df = spark.sql(f"""
            SELECT
              COALESCE(i.date_ym, s.date_ym) AS date_ym,
              COALESCE(i.mac_addr, s.mac_addr) AS mac_addr,
              COALESCE(i.X_Device_Country, s.X_Device_Country_speaker) AS X_Device_Country,
              COALESCE(i.X_Device_Sales_Model, s.X_Device_Sales_Model_speaker) AS X_Device_Sales_Model,
              i.log_time_inputlist,
              i.normal_log_inputlist,
              s.log_time_speaker,
              s.normal_log_speaker,
              CASE
                WHEN i.mac_addr IS NOT NULL AND s.mac_addr IS NOT NULL THEN 'both'
                WHEN i.mac_addr IS NOT NULL THEN 'only_inputlist'
                WHEN s.mac_addr IS NOT NULL THEN 'only_speaker'
              END AS source_type
            FROM (
              SELECT 
                date_ym,
                log_create_time AS log_time_inputlist,
                mac_addr,
                X_Device_Country,
                X_Device_Sales_Model,
                normal_log AS normal_log_inputlist
              FROM (
                SELECT *,
                      ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
                FROM kic_data_ods.tlamp.normal_log_{webos_version}
                WHERE date_ym BETWEEN '{date_ym}' AND '{date_ym}'
                  AND context_name = 'EIM'
                  AND message_id = 'NL_EIM_INPUTLIST'
                  AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
              ) sub
              WHERE rk = 1
            ) i
            FULL OUTER JOIN (
              SELECT 
                date_ym,
                log_create_time AS log_time_speaker,
                mac_addr,
                X_Device_Country AS X_Device_Country_speaker,
                X_Device_Sales_Model AS X_Device_Sales_Model_speaker,
                normal_log AS normal_log_speaker
              FROM (
                SELECT *,
                      ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
                FROM kic_data_ods.tlamp.normal_log_{webos_version}
                WHERE date_ym BETWEEN '{date_ym}' AND '{date_ym}'
                  AND context_name = 'com.palm.app.settings'
                  AND message_id = 'NL_SETTING_SPEAKER'
                  AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
              ) sub
              WHERE rk = 1
            ) s
            ON i.date_ym = s.date_ym AND i.mac_addr = s.mac_addr
          """)

    # 평탄화하기 
    flatten_df = mk_flatten_df(df)    
    # display(df)
    # 단일 CSV 파일로 저장 (볼륨 경로 지정)
    flatten_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(f"/Volumes/sandbox/z_yeswook_kim/v_yeswook_kim/heds/{webos_version}_{date_ym}_heds_914_csv")
    print(webos_version, date_ym,'ended')
    
    # del df
    # del flatten_df

In [0]:
import boto3
import os

import botocore
from datetime import datetime, timedelta
import urllib3

# AWS S3 클라이언트 생성
s3 = boto3.client(
    's3'
    # , aws_access_key_id=aws_personal_access_key['aws_access_key_id']
    # , aws_secret_access_key=aws_personal_access_key['aws_secret_access_key']
)

s3 = boto3.client('s3', config=botocore.client.Config(signature_version='s3v4'), verify=False)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)




# 시작 월과 종료 월 지정
start_month = "2024-01"
end_month = "2025-06"

# 월 리스트 생성
date_ym_list = []
current = datetime.strptime(start_month, "%Y-%m")
end = datetime.strptime(end_month, "%Y-%m")

while current <= end:
    date_ym_list.append(current.strftime("%Y-%m"))
    # 다음 달로 이동
    current += timedelta(days=32)
    current = current.replace(day=1)


# 반복 대상
webos_versions = ['webos22', 'webos23', 'webos24']
# date_ym_list = ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2025-06']

# S3 기본 정보
bucket = 's3-lge-he-sandbox-kic'
base_prefix = '__unitystorage/schemas/06a06837-4731-481a-80d3-140da5d8f193/volumes/64b39521-99cf-4829-bc2a-55313147ba19/heds'

# 로컬 저장 경로
local_dir = './downloaded_csvs'
os.makedirs(local_dir, exist_ok=True)

for webos_version in webos_versions[:]:
    for date_ym in date_ym_list[:]:
        prefix = f"{base_prefix}/{webos_version}_{date_ym}_heds_914_csv/"
        response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

        if 'Contents' not in response:
            print(f"No file found for {webos_version} {date_ym}")
            continue

        for obj in response['Contents']:
            key = obj['Key']
            if key.endswith('.csv') or 'part-' in key:
                local_filename = f"{webos_version}_{date_ym}_heds_914.csv"
                temp_filename = f"{webos_version}_{date_ym}_temp.csv"
                temp_path = os.path.join(local_dir, temp_filename)
                final_path = os.path.join(local_dir, local_filename)

                print(f"Downloading {key} to {temp_path}")
                s3.download_file(bucket, key, temp_path)

                # BOM 추가하여 저장
                with open(temp_path, 'r', encoding='utf-8') as f_in:
                    content = f_in.read()
                with open(final_path, 'w', encoding='utf-8-sig') as f_out:
                    f_out.write(content)

                os.remove(temp_path)
                print(f"{webos_version} {date_ym} 저장 완료 (BOM 포함)")
                break
        



In [0]:
%sql
SELECT
  'webos23' AS webos_version,  -- webOS 버전 추가
  COALESCE(i.date_ym, s.date_ym) AS date_ym,
  COALESCE(i.mac_addr, s.mac_addr) AS mac_addr,
  COALESCE(i.X_Device_Country, s.X_Device_Country_speaker) AS X_Device_Country,
  COALESCE(i.X_Device_Sales_Model, s.X_Device_Sales_Model_speaker) AS X_Device_Sales_Model,
  i.log_time_inputlist,
  i.normal_log_inputlist,
  s.log_time_speaker,
  s.normal_log_speaker,
  CASE
    WHEN i.mac_addr IS NOT NULL AND s.mac_addr IS NOT NULL THEN 'both'
    WHEN i.mac_addr IS NOT NULL THEN 'only_inputlist'
    WHEN s.mac_addr IS NOT NULL THEN 'only_speaker'
  END AS source_type
FROM (
  SELECT 
    date_ym,
    log_create_time AS log_time_inputlist,
    mac_addr,
    X_Device_Country,
    X_Device_Sales_Model,
    normal_log AS normal_log_inputlist
  FROM (
    SELECT *,
          ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
    FROM kic_data_ods.tlamp.normal_log_webos23
    WHERE date_ym BETWEEN '{date_ym}' AND '{date_ym}'
      AND context_name = 'EIM'
      AND message_id = 'NL_EIM_INPUTLIST'
      AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
  ) sub
  WHERE rk = 1
) i
FULL OUTER JOIN (
  SELECT 
    date_ym,
    log_create_time AS log_time_speaker,
    mac_addr,
    X_Device_Country AS X_Device_Country_speaker,
    X_Device_Sales_Model AS X_Device_Sales_Model_speaker,
    normal_log AS normal_log_speaker
  FROM (
    SELECT *,
          ROW_NUMBER() OVER (PARTITION BY date_ym, mac_addr ORDER BY log_create_time DESC) AS rk
    FROM kic_data_ods.tlamp.normal_log_webos23
    WHERE date_ym BETWEEN {date_ym} AND {date_ym}
      AND context_name = 'com.palm.app.settings'
      AND message_id = 'NL_SETTING_SPEAKER'
      AND X_Device_Country IN ('PH', 'AU', 'ID', 'MY', 'SG', 'TH', 'VN', 'TW', 'HK', 'JP', 'IN')
  ) sub
  WHERE rk = 1
) s
ON i.date_ym = s.date_ym AND i.mac_addr = s.mac_addr
